In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*/')

In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*/')

In [7]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [8]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [9]:
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
        .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
        .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [12]:
common_cols = []

green_cols = set(df_green.columns)
for col in green_cols:
    if col in df_yellow.columns:
        common_cols.append(col)

In [29]:
common_cols

['DOLocationID',
 'total_amount',
 'trip_distance',
 'congestion_surcharge',
 'PULocationID',
 'RatecodeID',
 'dropoff_datetime',
 'mta_tax',
 'tolls_amount',
 'passenger_count',
 'fare_amount',
 'improvement_surcharge',
 'store_and_fwd_flag',
 'pickup_datetime',
 'extra',
 'VendorID',
 'payment_type',
 'tip_amount']

In [13]:
from pyspark.sql import functions as F

In [14]:
df_green_ser = df_green \
                .select(common_cols) \
                .withColumn('service_type', F.lit('green'))

df_yellow_ser = df_yellow \
                .select(common_cols) \
                .withColumn('service_type', F.lit('yellow'))

In [15]:
df_trips_data = df_green_ser.unionAll(df_yellow_ser)

In [17]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [19]:
df_trips_data.createOrReplaceTempView('trips_data')

In [22]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM trips_data
GROUP BY 1
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [23]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [28]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')